In [1]:
import pandas as pd

In [2]:
from pypdf import PdfReader
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
data = pd.read_json("wikipedia_articles.json")

In [4]:
data.head()

,id,title,url,content
0,544195,Dours,https://fr.wikipedia.org/wiki?curid=544195,Dours\n\nDours (Dors en occitan) est une commu...
1,544196,Ens (Hautes-Pyrénées),https://fr.wikipedia.org/wiki?curid=544196,Ens (Hautes-Pyrénées)\n\nEns est une commune f...
2,544197,Esbareich,https://fr.wikipedia.org/wiki?curid=544197,Esbareich\n\nEsbareich est une commune françai...
3,544198,Escala,https://fr.wikipedia.org/wiki?curid=544198,Escala\n\nEscala est une commune française sit...
4,544199,Escaunets,https://fr.wikipedia.org/wiki?curid=544199,Escaunets\n\nEscaunets est une commune françai...


In [ ]:
print(data["content"][0])

In [5]:
def initialize_driver():
    options = Options()
    options.add_argument("--headless")  # Activer le mode headless
    options.add_argument("--no-sandbox")
    # driver = webdriver.Firefox(options=options)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = options)
    
    return driver

In [6]:
# Fonction pour diviser le texte en morceaux de 3900 caractères ou moins
def split_text(text, max_length=3900):
    # Diviser le texte en morceaux plus petits si la longueur dépasse la limite
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# Fonction de traduction
def translate_to_fon(text: str):
    driver = initialize_driver()
    driver.get("https://translate.google.com/details?hl=fr&sl=auto&tl=fon&op=translate")
    
    # Diviser le texte en morceaux de taille appropriée
    text_parts = split_text(text)
    translated_text = ""

    try:
        for part in text_parts:
            # Passer le texte dans la zone de saisie
            driver.find_element(by=By.CLASS_NAME, value="er8xn").send_keys(part) 

            
            # Attendre la traduction
            element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "HwtZe"))
            )
            
            # Récupérer la traduction pour ce morceau
            translated_text += element.text + " "  # Ajouter un espace entre les morceaux de texte

        return translated_text.strip()
    
    except Exception as e:
        print(f"Erreur : {e}")
        return None
    
    finally:
        driver.quit()

In [ ]:
translate_in_fon = translate_to_fon(data["content"][0])
print(translate_in_fon)

In [ ]:
import time
import os

# ==============================
# CONFIG
BATCH_SIZE = 10
SLEEP_BETWEEN = 10  # secondes
CSV_INPUT = "extracted_data.csv"
CSV_OUTPUT = "translated_data_fon.csv"
# ==============================

# Si un fichier de sortie existe, on le reprend pour éviter de tout refaire
if os.path.exists(CSV_OUTPUT):
    df_translated = pd.read_csv(CSV_OUTPUT)
    already_done_ids = set(df_translated["id"])
    print(f"🔁 Reprise : {len(already_done_ids)} articles déjà traduits.")
else:
    df_translated = pd.DataFrame(columns=["id", "title", "url", "content", "content_fon"])
    already_done_ids = set()


try:
    for idx, row in data.iterrows():
        if row["id"] in already_done_ids:
            continue  # Déjà traité

        print(f"➡️  [{idx+1}] Traduction de : {row['title']}")
        translated = translate_to_fon(row["content"])
        new_row = {
            "id": row["id"],
            "title": row["title"],
            "url": row["url"],
            "content": row["content"],
            "content_fon": translated
        }
        df_translated = pd.concat([df_translated, pd.DataFrame([new_row])], ignore_index=True)

        # Sauvegarder toutes les BATCH_SIZE traductions
        if len(df_translated) % BATCH_SIZE == 0:
            df_translated.to_csv(CSV_OUTPUT, index=False)
            print(f"💾 Sauvegarde automatique après {len(df_translated)} articles.")
            time.sleep(SLEEP_BETWEEN)  # Pause entre les lots

    # Sauvegarde finale
    df_translated.to_csv(CSV_OUTPUT, index=False)
    print("✅ Traduction terminée. Fichier final : translated_data_fon.csv")

except KeyboardInterrupt:
    print("⏹️ Interruption manuelle, sauvegarde en cours...")
    df_translated.to_csv(CSV_OUTPUT, index=False)
    print("✅ Progression sauvegardée.")



🔁 Reprise : 5180 articles déjà traduits.
➡️  [5181] Traduction de : Léonce Perret
